In [17]:
def api():
    import pygsheets
    import pandas as pd
    import warnings
    
    warnings.filterwarnings("ignore")

    gc = pygsheets.authorize(service_file='pinpongironhack-601c5f87ad39.json')

    sh = gc.open('Ping Pong Ironhack (respuestas)')

    sh[0]

    data = sh[0].get_all_values()

    marcatemporal = []
    player1 = []
    score1 = []
    player2 = []
    score2 = []
    almejorde = []

    for row in data:
        marcatemporal.append(row[0])
        player1.append(row[1])
        score1.append(row[2])
        player2.append(row[3])
        score2.append(row[4])
        almejorde.append(row[5])

    data = {}

    data["marcatemporal"] = marcatemporal
    data["player1"] = player1
    data["score1"] = score1
    data["player2"] = player2
    data["score2"] = score2
    data["almejorde"] = almejorde

    import pandas as pd

    data = pd.DataFrame(data)[1:]
    data['player1'] = data['player1'].str.replace('Folch','Marc')
    data['player2'] = data['player2'].str.replace('Folch','Marc')
    data

    database = {}
    
    for i in data.iterrows():
        if (i[1][1] not in database.keys()) and (len(i[1][1]) != 0):
            database[i[1][1]] = 1000
        if (i[1][3] not in database.keys()) and (len(i[1][3]) != 0):
            database[i[1][3]] = 1000
    
    def elo(a, b, winner):
        
        a1 = 10 ** ( database[a] / 400 )
        b1 = 10 ** ( database[b] / 400 )

        a2 = a1 / ( a1 + b1 )
        b2 = b1 / ( a1 + b1 )

        if winner == 1:
            a3 = round( database[a] + ( 32 * ( 1 - a2 ) ) )
            b3 = round( database[b] + ( 32 * ( 0 - b2 ) ) )
            
        elif winner == 2:
            a3 = round( database[a] + ( 32 * ( 0 - a2 ) ) )
            b3 = round( database[b] + ( 32 * ( 1 - b2 ) ) )

        database[a] = a3
        database[b] = b3

    for i in data.iterrows():
        if (i[1][1]) != "" and (i[1][3]) != "":
            if (i[1][2]) > (i[1][4]):
                x = 1
            else:
                x = 2
            
            elo((i[1][1]), (i[1][3]), x)

    names = []
    elos = []

    for i, j in database.items():
        names.append(i)
        elos.append(j)

    thetable = {}

    thetable["Name"] = names
    thetable["ELO"] = elos

    table = pd.DataFrame(thetable).sort_values("ELO", ascending=False).reset_index().drop(["index"], axis=1)
    table

    html_table = table.to_html()

    with open('table.html', 'w') as f:
        f.write(html_table)

    return table

In [18]:
api()

,Name,ELO
0,Miki,1046
1,Marc,1044
2,Marco,1016
3,Ricardo,985
4,Lucia,985
5,Amir,985
6,Víctor,972
7,Uri,967
